In [1]:
import pandas as pd
import numpy as np
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [21]:
wm_inventory_ds = pd.read_csv('/content/drive/MyDrive/Software_Tools/ReAllocate_Guest_Tool/src/2024_nov_wm_inventory_reservations_details(wm).csv',encoding='ISO-8859-1')
wm_inventory_ds['resort'] = wm_inventory_ds['resort'].str.replace('WorldMark', '').str.strip()
#wm_inventory_ds['confirmation'] = wm_inventory_ds[' confirmation '].str.strip()
#wm_inventory_ds.drop(columns=[' confirmation '],inplace = True)

In [22]:
import re
def infer_unit_type(unit_description):
    # Convert unit_description to string to handle non-string values like numbers or NaN
    unit_description = str(unit_description)

    # Search for a number followed by "Bedroom" in the unit description
    match = re.search(r'(\d+)\s*Bedroom', unit_description, re.IGNORECASE)
    if match:
        return float(match.group(1))
    elif 'studio' in unit_description.lower():
        return 1.0
    else:
        return np.nan

In [23]:
batch_set = wm_inventory_ds[['confirmation',
                             'reservationId',
                             'account',
                             'resort',
                             'checkin',
                             'checkout',
                             'cancel_by',
                             'Unit',
                             'credits',
                             'rented?',
                             'listing?',
                             'cost basis',
                             'target payout',
                             'nights',
                             'booking_date']]

batch_set['Unit Type'] = batch_set['Unit'].apply(infer_unit_type)

In [24]:
import pandas as pd
import numpy as np
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error
from sklearn.preprocessing import LabelEncoder
from datetime import datetime, timedelta
import warnings
warnings.filterwarnings('ignore')

def get_us_holidays(year):
    return {
        "New Year's Day": datetime(year, 1, 1),
        "Independence Day": datetime(year, 7, 4),
        "Thanksgiving": datetime(year, 11, 28),
        "Christmas": datetime(year, 12, 25),
        "Labor Day": datetime(year, 9, 2),
        "Memorial Day": datetime(year, 5, 27),
        "Veterans Day": datetime(year, 11, 11),
        "Martin Luther King Jr. Day": datetime(year, 1, 15),
        "Columbus Day": datetime(year, 10, 12),
        "Washingtons Birthday": datetime(year, 2, 18),
        "Presidents Day": datetime(year, 2, 19) ,
        "Cochela_Festival_w_1":datetime(year, 4, 12),
        "Cochela_Festival_w_2":datetime(year, 4, 19),
        "Cochela_Festival_w_3":datetime(year, 4, 26),
        "Oktober_Festival_w_1": datetime(year, 10, 11),
        "Oktober_Festival_w_2":datetime(year, 4,18),
        "Oktober_Festival_w_3":datetime(year, 4, 25)

    }


def is_holiday_overlap(checkin, checkout):
    if pd.isnull(checkin) or pd.isnull(checkout):
        return 0
    holidays = get_us_holidays(checkin.year)
    for holiday in holidays.values():
        if checkin <= holiday <= checkout:
            return 1
    return 0




batch_set['checkin'] = pd.to_datetime(batch_set['checkin'], format='%d-%b-%y')
batch_set['checkout'] = pd.to_datetime(batch_set['checkout'], format='%d-%b-%y')
batch_set['booking_date'] = pd.to_datetime(batch_set['booking_date'], format='%d-%b-%y', errors='coerce')
batch_set['booking_date'] = batch_set['booking_date'].fillna(
    pd.to_datetime(batch_set['booking_date'], format='%y-%b-%d', errors='coerce')
)
def parse_dates(date_str):
    for fmt in ('%d-%b-%y', '%m/%d/%Y', '%d/%m/%Y'):
        try:
            return pd.to_datetime(date_str, format=fmt)
        except ValueError:
            continue
    return pd.NaT  # Return NaT if no format matches

#training_set['Start date'] = training_set['Start date'].apply(parse_dates)
#training_set['End date'] = training_set['End date'].apply(parse_dates)
#training_set['Booked'] = pd.to_datetime(training_set['Booked'], format='%d-%b-%y', errors='coerce')
#training_set['Booked'] = training_set['Booked'].fillna(
#    pd.to_datetime(training_set['Booked'], format='%y-%b-%d', errors='coerce')
#)
batch_set['stay_duration'] = (batch_set['checkout'] - batch_set['checkin']).dt.days
batch_set['has_weekend'] = batch_set['checkin'].apply(lambda x: 1 if x.weekday() >= 5 else 0)
batch_set['is_holiday_season'] = batch_set['checkin'].apply(lambda x: 1 if x.month in [12, 1] else 0)
batch_set['holiday_overlap'] = batch_set.apply(
    lambda row: is_holiday_overlap(row['checkin'], row['checkout']), axis=1
)
batch_set['booking_advance_days'] = (batch_set['checkin'] - batch_set['booking_date']).dt.days
batch_set['checkin_year'] = batch_set['checkin'].dt.year

#training_set['stay_duration'] = (training_set['End date'] - training_set['Start date']).dt.days
#training_set['booking_advance_days'] = (training_set['Start date'] - training_set['Booked']).dt.days
#training_set['checkin_year'] = training_set['Start date'].dt.year
#training_set['has_weekend'] = training_set['Start date'].apply(lambda x: 1 if x.weekday() >= 5 else 0)
#training_set['is_holiday_season'] = training_set['Start date'].apply(lambda x: 1 if x.month in [12, 1] else 0)
#training_set['holiday_overlap'] = training_set.apply(
#    lambda row: is_holiday_overlap(row['Start date'], row['End date']), axis=1
#)

In [25]:
encoder = LabelEncoder()
batch_set['resort'] = encoder.fit_transform(batch_set['resort'])
batch_set.rename(columns={'resort':'Location','nights':'# of nights'}, inplace= True)

In [26]:
##select request data

batch_set = batch_set[~pd.notnull(batch_set['rented?'])]
confirmation_codes = batch_set.confirmation.values
features = batch_set[['Location', 'has_weekend', 'is_holiday_season', '# of nights', 'holiday_overlap', 'Unit Type', 'checkin_year']]

In [8]:
import pickle
with open('/content/drive/MyDrive/Software_Tools/Pricing_Tool/models/Pricing_XGB_69.pkl', 'rb') as file:
    loaded_model = pickle.load(file)

In [27]:
log_predictions_xgb = loaded_model.predict(features)
original_scale_predictions_xgb = np.expm1(log_predictions_xgb)

In [28]:
Pricing = pd.DataFrame({'confirmation': confirmation_codes, 'Price': original_scale_predictions_xgb})
Pricing['Price'] = Pricing['Price'] * (1)

In [29]:
Pricing[Pricing.confirmation == 'RWO0042720878BR']

,confirmation,Price
4,RWO0042720878BR,1633.445435
